In [13]:
import os
from mask_function import get_mask
import cv2
import pandas as pd
import numpy as np
from feature_A import feature_A
from feature_B import feature_B
from feature_C import feature_C
from feature_H import feature_H

In [14]:
## Fetch files in directories, create base paths
images=os.listdir(r'C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\images')
masks=os.listdir(r'C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\padchest_lesion_masks\lesion_masks')
image_path=r'C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\images'
mask_path=r'C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\padchest_lesion_masks\lesion_masks'

## Remove duplicate masks
images_set=set(images)
masks=[mask for mask in masks if mask[:-9]+'.png' in images_set]

## Load CSV and create a column for presence of cancer
df_ground=pd.read_csv(r"C:\Users\MATEJ\Documents\BDS SEMESTER 2\Projects in Data Science\Project\metadata.csv")
df_ground['cancer']=np.where( (df_ground[ 'diagnostic']=='BCC') ^ (df_ground['diagnostic']=='MEL') ^ (df_ground['diagnostic']=='SCC'),1,0)

## Create empty df for features
df=pd.DataFrame()

In [15]:
## Main loop through pictures
image_idx = 0
mask_idx = 0
has_mask = True

while image_idx < 200: # and mask_idx < len(masks): ## len(images)
    image = images[image_idx]
    mask = masks[mask_idx]
    
    expected_im = mask[:-9] + '.png' ## Remove _mask from mask name, compare to image

    if image == expected_im: ## If matching, load both
        has_mask = True
        mask_idx += 1  ## Move to next mask only if match
        image=cv2.imread(f'{image_path}/{image}',cv2.IMREAD_COLOR_RGB)
        mask=cv2.imread(f'{mask_path}/{mask}',cv2.IMREAD_GRAYSCALE) ## TEMP FIX
        if np.sum(np.nonzero(mask))==0: ## If mask black image
            mask=get_mask(image) ## Make own mask

    else: ## If not matching, load image and create mask
        has_mask = False
        image=cv2.imread(f'{image_path}/{image}',cv2.IMREAD_COLOR_RGB)
        mask=get_mask(image)

    ## Change (potential) boolean mask to int
    mask=mask.astype(int)
    ## Mask image with mask
    image_masked=image.copy()
    image_masked[mask==0] = 0

    ## Extract features from images/masks
    feature_A(image_masked,mask,image_idx,df)
    feature_B(image_masked,mask,image_idx,df)
    feature_C(image_masked,mask,image_idx,df)
    feature_H(image_masked,image_idx,df)
    image_idx += 1  ## Always move to the next image
df['Cancer']=df_ground['cancer']

c:\Users\MATEJ\anaconda3\Lib\site-packages\skimage\segmentation\slic_superpixels.py:60: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  centroids, _ = kmeans2(coord[idx_dense], coord[idx], iter=5)
c:\Users\MATEJ\anaconda3\Lib\site-packages\skimage\segmentation\slic_superpixels.py:60: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  centroids, _ = kmeans2(coord[idx_dense], coord[idx], iter=5)


feature_A - worst - 6m \
feature_B - compactness - 26s \
feature_C - rgb_var - 10m, so bad \
feature_H - 1m30s \
total 17 mins with 200 pics

In [16]:
pd.DataFrame.to_csv(df,'features_model.csv',index_label='image_id')
df

,A - worst asymmetry,B - compactness,C - red variance,C - green variance,C - blue variance,Hair,Cancer
0,4.296189e+06,-2.184952,0.107957,0.197003,0.243376,0.052947,0
1,7.772916e+06,162.521104,0.006297,0.017192,0.027633,0.016712,1
2,7.992507e+06,-174.677551,0.010703,0.015358,0.019743,0.035240,0
3,2.117490e+06,3.893275,6.593656,8.764123,10.345878,0.174942,0
4,8.580224e+08,30.588197,74.546797,85.081084,86.207771,0.083847,1
...,...,...,...,...,...,...,...
195,1.635118e+06,9.758157,0.006080,0.008644,0.009565,0.005090,1
196,3.528478e+06,0.397130,23.067684,29.442379,41.735519,0.110329,1
197,2.732572e+06,21.576770,1.650222,3.203600,3.678853,0.049630,0
198,2.151558e+06,-8.514855,1.262599,3.057201,3.562581,0.086601,0
